In [1]:
from dotenv import load_dotenv
import os
import requests
from typing import Any, Dict

load_dotenv()


def google_search(query:str) -> dict[str,Any]:
    """
    Performs a Google search and returns a list of results.
    :param query: The search query string.
    :param api_key: Your Google API key.
    :param cse_id: Your Custom Search Engine ID.
    :param num_results: Number of results to return (default: 5).
    :return: List of search result dictionaries.
    """
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,
        "key": os.getenv("GOOGLE_WEATHER_API_KEY"),
        "cx": os.getenv("GOOGLE_SEARCH_CSE_ID"),
        "num": 5,
    }
    response = requests.get(url, params=params)
    print(response.status_code, response.text)
    results = response.json().get("items", [])

    hmm_res = [
        {
            "title": item.get("title"),
            "snippet": item.get("snippet"),
            "link": item.get("link"),
        }
        for item in results
    ]
    print(hmm_res)
    str_res = "\n".join(
        f"{i + 1}. {item['title']}\n{item['snippet']}\n{item['link']}"
        for i, item in enumerate(hmm_res)
    )
    final_res = {
        "res":str_res if str_res else "No results found."
    }
    return final_res

In [2]:
params = {
        "key": os.getenv("GOOGLE_WEATHER_API_KEY"),
        "cx": os.getenv("GOOGLE_SEARCH_CSE_ID"),
        "num": 10,
    }

params

{'key': '4c5eb0c6164a7ac1ae87cb3df5e78414',
 'cx': '47fa5cf89c7df43e2',
 'num': 10}

In [3]:
google_search("Who is Rishub C R?")

400 {
  "error": {
    "code": 400,
    "message": "API key not valid. Please pass a valid API key.",
    "errors": [
      {
        "message": "API key not valid. Please pass a valid API key.",
        "domain": "global",
        "reason": "badRequest"
      }
    ],
    "status": "INVALID_ARGUMENT",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.ErrorInfo",
        "reason": "API_KEY_INVALID",
        "domain": "googleapis.com",
        "metadata": {
          "service": "customsearch.googleapis.com"
        }
      },
      {
        "@type": "type.googleapis.com/google.rpc.LocalizedMessage",
        "locale": "en-US",
        "message": "API key not valid. Please pass a valid API key."
      }
    ]
  }
}

[]


{'res': 'No results found.'}

In [4]:
import json
import requests

async def get_soilgrids_data(lat: float, lon: float) -> dict:
    """Fetches soil property data for the given coordinates using the SoilGrids v2.0 REST API."""
    properties = [
        "bdod",
        "cec",
        "cfvo",
        "clay",
        "nitrogen",
        "ocd",
        "ocs",
        "phh2o",
        "sand",
        "silt",
        "soc",
        "wv0010",
        "wv0033",
        "wv1500"
    ]
    depths = ["0-5cm", "0-30cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]
    values = ["Q0.05", "Q0.5", "Q0.95", "mean", "uncertainty"]

    try:
        url = f"https://rest.isric.org/soilgrids/v2.0/properties/query?lon={lon}&lat={lat}"
        for prop in properties:
            url += f"&property={prop}"
        for depth in depths:
            url += f"&depth={depth}"
        for value in values:
            url += f"&value={value}"

        response = requests.get(url, timeout=200, headers={"accept": "application/json"})
        response.raise_for_status()

        soil_data = response.json()
        processed_data = {}

        # Updated processing logic to match the actual API response structure
        if "properties" in soil_data and "layers" in soil_data["properties"]:
            for layer in soil_data["properties"]["layers"]:
                prop_name = layer.get("name")
                if prop_name:
                    processed_data[prop_name] = {}
                    # Add unit information
                    if "unit_measure" in layer:
                        processed_data[prop_name]["unit_measure"] = layer["unit_measure"]
                    
                    # Process depths
                    for depth_info in layer.get("depths", []):
                        depth_label = depth_info.get("label", "unknown")
                        depth_values = depth_info.get("values", {})
                        processed_data[prop_name][depth_label] = depth_values

        return {
            "status": "success",
            "data": json.dumps(processed_data)
        }

    except Exception as e:
        return {
            "status": "error",
            "error_message": f"Failed to fetch SoilGrids data: {str(e)}",
        }

In [5]:
await get_soilgrids_data(12.9716, 77.5946)  # Example coordinates for Bangalore

{'status': 'success',
 'data': '{"bdod": {"unit_measure": {"d_factor": 100, "mapped_units": "cg/cm\\u00b3", "target_units": "kg/dm\\u00b3", "uncertainty_unit": ""}, "0-5cm": {"Q0.05": null, "Q0.5": null, "Q0.95": null, "mean": null, "uncertainty": null}, "5-15cm": {"Q0.5": null, "Q0.05": null, "Q0.95": null, "mean": null, "uncertainty": null}, "15-30cm": {"Q0.05": null, "Q0.5": null, "Q0.95": null, "mean": null, "uncertainty": null}, "30-60cm": {"Q0.05": null, "Q0.5": null, "Q0.95": null, "mean": null, "uncertainty": null}, "60-100cm": {"Q0.05": null, "Q0.5": null, "Q0.95": null, "mean": null, "uncertainty": null}, "100-200cm": {"Q0.5": null, "Q0.05": null, "Q0.95": null, "mean": null, "uncertainty": null}}, "cec": {"unit_measure": {"d_factor": 10, "mapped_units": "mmol(c)/kg", "target_units": "cmol(c)/kg", "uncertainty_unit": ""}, "0-5cm": {"Q0.5": null, "Q0.05": null, "Q0.95": null, "mean": null, "uncertainty": null}, "5-15cm": {"Q0.5": null, "Q0.05": null, "Q0.95": null, "mean": nul

In [6]:
from google.adk.tools import google_search


def create_google_search_tool():
    return google_search


import requests
import os
from typing import Dict, Any


def get_current_weather(lat: float, lon: float) -> Dict[str, Any]:
    api_key = os.getenv("GOOGLE_WEATHER_API_KEY")
    endpoint = "https://maps.googleapis.com/maps/api/weather/v1/current"
    params = {"location": f"{lat},{lon}", "key": api_key}
    resp = requests.get(endpoint, params=params)
    if resp.status_code == 200:
        return resp.json()
    return {"error": resp.text, "status_code": resp.status_code}


import requests
from typing import Dict, Any


def get_soilgrids_data(lat: float, lon: float) -> Dict[str, Any]:
    """
    Fetches soil property data for the given latitude and longitude using the SoilGrids REST API.
    Returns surface (0-5cm) values for all soil properties.
    """
    url = f"https://rest.soilgrids.org/query?lon={lon}&lat={lat}"
    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        soil_data = resp.json()
        surface_props = {}
        properties = soil_data.get("properties", {})
        for prop, v in properties.items():
            try:
                surface_val = v["values"][0]["mean"]
                surface_props[prop] = surface_val
            except Exception:
                continue
        return {
            "status": "success",
            "latitude": lat,
            "longitude": lon,
            "surface_soil_properties": surface_props,
            "soilgrids_raw": properties,
        }
    except Exception as e:
        return {
            "status": "error",
            "message": f"Failed to fetch SoilGrids data: {str(e)}",
        }

In [7]:
# 12.9629° N, 77.5775° E
get_current_weather(12.9629, 77.5775)  # Example coordinates for Bangalore

{'error': '<!DOCTYPE html>\n<html lang=en>\n  <meta charset=utf-8>\n  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">\n  <title>Error 404 (Not Found)!!1</title>\n  <style>\n    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) no-repeat 0% 0%/100% 100%;-moz-border

In [21]:
import requests


def get_weather_data(lat, lon, api_key):
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"

    response = requests.get(url, params=params)
    return response.json()

In [22]:
get_weather_data(12.9629, 77.5775, "4c5eb0c6164a7ac1ae87cb3df5e78414")  # Example coordinates for Bangalore

{'cod': 401,
 'message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}

In [ ]:
from google.adk.agents import Agent

root_agent = Agent(
    name="multi_api_streaming_agent",
    model="gemini-2.0-flash-live-001",
    description=(
        "Expert assistant with access to Google Search, Location, Weather, and Soil Data APIs. "
        "Given a location (as name/address or lat/lon), can provide coordinates, weather, and full soil properties."
    ),
    instruction="""You are an advanced assistant for environmental and location data.
    - Given a user-input location (address or latitude/longitude), you can fetch coordinates, current weather,
      and a detailed set of soil properties for that point using SoilGrids.
    - For soil properties, call the 'get_soilgrids_data' tool when lat/lon are specified.
    - Respond with clear, concise information and highlight key findings (such as soil pH, nitrogen, sand/clay content, etc).
    """,
    tools=[google_search, get_current_weather, get_soilgrids_data],
)

In [ ]:
from google.adk.orchestration.session import InMemorySessionService
from google.generativeai.types.content_types import Content, Part

user_id = "demo_user"
session_id = "demo_session"
query = "Give me soil and weather data for Bengaluru, India"

# Build Content object
content = Content(role="user", parts=[Part.from_text(query)])

# Run the agent (synchronous/simple mode)
result = root_agent.run_sync(user_id=user_id, session_id=session_id, content=content)
print(result.text())